# Problem Statement 

    -> Our goal is to predict the sale price of house for the given id

# Import Libraries

In [29]:
import numpy as numpy
import pandas as pd

from sklearn.preprocessing import StandardScaler,OrdinalEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Data Loading

In [4]:
def data_loader(path):
    df = pd.read_csv(path)
    return df

In [16]:
data =data_loader('../data/cleaned.csv')
data

,Unnamed: 0,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1455,1456,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,1456,1457,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,...,0,0,0,0,0,2,2010,WD,Normal,210000
1457,1457,1458,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,...,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,1458,1459,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,...,112,0,0,0,0,4,2010,WD,Normal,142125


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 76 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     1460 non-null   int64  
 1   Id             1460 non-null   int64  
 2   MSSubClass     1460 non-null   int64  
 3   MSZoning       1460 non-null   object 
 4   LotFrontage    1460 non-null   float64
 5   LotArea        1460 non-null   int64  
 6   Street         1460 non-null   object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [25]:
# Splitting data 

x = data.drop(['SalePrice'],axis=1)
y = data['SalePrice']



In [39]:
con = x.select_dtypes(exclude='object').columns
cat = x.select_dtypes(include='object').columns

num_pipe = Pipeline(steps=([('Impute',SimpleImputer(strategy='median')),('Scalar',StandardScaler())]))
cat_pipe = Pipeline(steps=([('Impute',SimpleImputer(strategy='most_frequent')),('Encoding',OrdinalEncoder())]))

pre = ColumnTransformer([('num_pipe',num_pipe,con),('cat_pipe',cat_pipe,cat)])

x1=pd.DataFrame(pre.fit_transform(x),columns=pre.get_feature_names_out())
x1

,num_pipe__Unnamed: 0,num_pipe__Id,num_pipe__MSSubClass,num_pipe__LotFrontage,num_pipe__LotArea,num_pipe__OverallQual,num_pipe__OverallCond,num_pipe__YearBuilt,num_pipe__YearRemodAdd,num_pipe__MasVnrArea,...,cat_pipe__Electrical,cat_pipe__KitchenQual,cat_pipe__Functional,cat_pipe__GarageType,cat_pipe__GarageFinish,cat_pipe__GarageQual,cat_pipe__GarageCond,cat_pipe__PavedDrive,cat_pipe__SaleType,cat_pipe__SaleCondition
0,-1.730865,-1.730865,0.073375,-0.220875,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.514104,...,4.0,2.0,6.0,1.0,1.0,4.0,4.0,2.0,8.0,4.0
1,-1.728492,-1.728492,-0.872563,0.460320,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.570750,...,4.0,3.0,6.0,1.0,1.0,4.0,4.0,2.0,8.0,4.0
2,-1.726120,-1.726120,0.073375,-0.084636,0.073480,0.651479,-0.517200,0.984752,0.830215,0.325915,...,4.0,2.0,6.0,1.0,1.0,4.0,4.0,2.0,8.0,4.0
3,-1.723747,-1.723747,0.309859,-0.447940,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.570750,...,4.0,2.0,6.0,5.0,2.0,4.0,4.0,2.0,8.0,0.0
4,-1.721374,-1.721374,0.073375,0.641972,0.375148,1.374795,-0.517200,0.951632,0.733308,1.366489,...,4.0,2.0,6.0,1.0,1.0,4.0,4.0,2.0,8.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1.721374,1.721374,0.073375,-0.357114,-0.260560,-0.071836,-0.517200,0.918511,0.733308,-0.570750,...,4.0,3.0,6.0,1.0,1.0,4.0,4.0,2.0,8.0,4.0
1456,1.723747,1.723747,-0.872563,0.687385,0.266407,-0.071836,0.381743,0.222975,0.151865,0.087911,...,4.0,3.0,2.0,1.0,2.0,4.0,4.0,2.0,8.0,4.0
1457,1.726120,1.726120,0.309859,-0.175462,-0.147810,0.651479,3.078570,-1.002492,1.024029,-0.570750,...,4.0,2.0,6.0,1.0,1.0,4.0,4.0,2.0,8.0,4.0
1458,1.728492,1.728492,-0.872563,-0.084636,-0.080160,-0.795151,0.381743,-0.704406,0.539493,-0.570750,...,0.0,2.0,6.0,1.0,2.0,4.0,4.0,2.0,8.0,4.0


In [40]:
lr = LinearRegression()
sfs = SequentialFeatureSelector(lr)

In [43]:
x2 = pd.DataFrame(sfs.fit_transform(x1,y),columns = sfs.get_feature_names_out())
x2

,num_pipe__Unnamed: 0,num_pipe__Id,num_pipe__MSSubClass,num_pipe__LotArea,num_pipe__OverallQual,num_pipe__OverallCond,num_pipe__YearBuilt,num_pipe__MasVnrArea,num_pipe__BsmtFinSF1,num_pipe__GrLivArea,...,cat_pipe__Exterior1st,cat_pipe__ExterQual,cat_pipe__BsmtQual,cat_pipe__BsmtCond,cat_pipe__BsmtExposure,cat_pipe__HeatingQC,cat_pipe__KitchenQual,cat_pipe__Functional,cat_pipe__GarageCond,cat_pipe__PavedDrive
0,-1.730865,-1.730865,0.073375,-0.207142,0.651479,-0.517200,1.050994,0.514104,0.575425,0.370333,...,12.0,2.0,2.0,3.0,3.0,0.0,2.0,6.0,4.0,2.0
1,-1.728492,-1.728492,-0.872563,-0.091886,-0.071836,2.179628,0.156734,-0.570750,1.171992,-0.482512,...,8.0,3.0,2.0,3.0,1.0,0.0,3.0,6.0,4.0,2.0
2,-1.726120,-1.726120,0.073375,0.073480,0.651479,-0.517200,0.984752,0.325915,0.092907,0.515013,...,12.0,2.0,2.0,3.0,2.0,0.0,2.0,6.0,4.0,2.0
3,-1.723747,-1.723747,0.309859,-0.096897,0.651479,-0.517200,-1.863632,-0.570750,-0.499274,0.383659,...,13.0,3.0,3.0,1.0,3.0,2.0,2.0,6.0,4.0,2.0
4,-1.721374,-1.721374,0.073375,0.375148,1.374795,-0.517200,0.951632,1.366489,0.463568,1.299326,...,12.0,2.0,2.0,3.0,0.0,0.0,2.0,6.0,4.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1.721374,1.721374,0.073375,-0.260560,-0.071836,-0.517200,0.918511,-0.570750,-0.973018,0.250402,...,12.0,3.0,2.0,3.0,3.0,0.0,3.0,6.0,4.0,2.0
1456,1.723747,1.723747,-0.872563,0.266407,-0.071836,0.381743,0.222975,0.087911,0.759659,1.061367,...,9.0,3.0,2.0,3.0,3.0,4.0,3.0,2.0,4.0,2.0
1457,1.726120,1.726120,0.309859,-0.147810,0.651479,3.078570,-1.002492,-0.570750,-0.369871,1.569647,...,5.0,0.0,3.0,1.0,3.0,0.0,2.0,6.0,4.0,2.0
1458,1.728492,1.728492,-0.872563,-0.080160,-0.795151,0.381743,-0.704406,-0.570750,-0.865548,-0.832788,...,8.0,3.0,3.0,3.0,2.0,2.0,2.0,6.0,4.0,2.0


In [44]:
# Splitting data for training and testing

x_train, x_test, y_train, y_test = train_test_split(x2,y,test_size=0.2,random_state=21)

In [51]:
# Model

line_reg = LinearRegression()

line_reg.fit(x2,y)

line_reg.intercept_

np.float64(188314.97810319695)

In [52]:
tr_pred = line_reg.predict(x_train)

In [53]:
def Evaluate(y_pred,y_actual):
    mae = mean_absolute_error(y_pred,y_actual)
    mse = mean_squared_error(y_pred,y_actual)
    rmse = numpy.sqrt(mse)
    r2 = r2_score(y_pred,y_actual)

    print('mse: ',mse)
    print('rmse:',rmse)
    print('mae:', mae)
    print('r2:', r2)


In [54]:
# Training Evaluation

Evaluate(tr_pred,y_train)

mse:  932129591.1488279
rmse: 30530.797420781986
mae: 19025.372772539424
r2: 0.8217906191070431


In [56]:
# Testing Evaluation

ts_pred = line_reg.predict(x_test)

Evaluate(ts_pred,y_test)

mse:  1223357480.55704
rmse: 34976.527565741
mae: 19452.263459610505
r2: 0.783555642395686


# Lasso Regression

In [57]:
la = Lasso()

In [58]:
la.fit(x2,y)

Lasso()

In [59]:
tr_y = la.predict(x_train)
Evaluate(tr_pred,y_train)

mse:  932129591.1488279
rmse: 30530.797420781986
mae: 19025.372772539424
r2: 0.8217906191070431


In [80]:
ts_y = la.predict(x_test)
Evaluate(ts_y,y_test)

mse:  1223297436.3935497
rmse: 34975.66920580005
mae: 19451.712383994276
r2: 0.7835531193958238


## Hyper Parameter Tuning

# GridSearch CV

In [61]:
grid = {
    'alpha': numpy.arange(0.01,3,0.01)
}

In [62]:
gs = GridSearchCV(la,param_grid=grid,cv=2)
gs.fit(x_train,y_train)

GridSearchCV(cv=2, estimator=Lasso(),
             param_grid={'alpha': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0....
       2.21, 2.22, 2.23, 2.24, 2.25, 2.26, 2.27, 2.28, 2.29, 2.3 , 2.31,
       2.32, 2.33, 2.34, 2.35, 2.36, 2.37, 2.38, 2.39, 2.4 , 2.41, 2.42,
       2.43, 2.44, 2.45, 2.46, 2.47, 2.48, 2.49, 2.5 , 2.51, 2.52, 2.53,
       2.54, 2.55, 2.56, 2.57, 2.58, 2.59, 2.6 , 2.61, 2.62, 2.63, 2.64,
       2.65, 2.66, 2.67, 2.68, 2.69, 2.7 , 2.71, 2.72, 2.73, 2.74, 2.75,
       2.76, 2.77, 2.78, 2.79, 2.8 , 2.81, 2.82, 2.83, 2.84, 2.85, 2.86,
       2.87, 2.88, 2.89, 2.9 , 2.91, 2.92, 2.93, 2.94, 2.95, 2.96, 2.97,
       2.98, 2.99])})

In [64]:
la1 = gs.best_estimator_

In [65]:
pred_t = la1.predict(x_train)

Evaluate(pred_t,y_train)

mse:  923811208.0266837
rmse: 30394.262748530087
mae: 18882.965751196683
r2: 0.8244935375857159


In [66]:
pred_ts = la1.predict(x_test)

Evaluate(pred_ts,y_test)

mse:  1298575247.2678301
rmse: 36035.74957272056
mae: 19888.148376396955
r2: 0.770401783817181


 # RandomSearch Cv

In [67]:
ra = RandomizedSearchCV(la,param_distributions=grid,cv=2)
ra.fit(x_train,y_train)

RandomizedSearchCV(cv=2, estimator=Lasso(),
                   param_distributions={'alpha': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0....
       2.21, 2.22, 2.23, 2.24, 2.25, 2.26, 2.27, 2.28, 2.29, 2.3 , 2.31,
       2.32, 2.33, 2.34, 2.35, 2.36, 2.37, 2.38, 2.39, 2.4 , 2.41, 2.42,
       2.43, 2.44, 2.45, 2.46, 2.47, 2.48, 2.49, 2.5 , 2.51, 2.52, 2.53,
       2.54, 2.55, 2.56, 2.57, 2.58, 2.59, 2.6 , 2.61, 2.62, 2.63, 2.64,
       2.65, 2.66, 2.67, 2.68, 2.69, 2.7 , 2.71, 2.72, 2.73, 2.74, 2.75,
       2.76, 2.77, 2.78, 2.79, 2.8 , 2.81, 2.82, 2.83, 2.84, 2.85, 2.86,
       2.87, 2.88, 2.89, 2.9 , 2.91, 2.92, 2.93, 2.94, 2.95, 2.96, 2.97,
       2.98, 2.99])})

In [68]:
ra1 = ra.best_estimator_

In [69]:
ra1

Lasso(alpha=np.float64(2.75))

In [70]:
pr = ra1.predict(x_train)

Evaluate(pr,y_train)

mse:  923809116.1583856
rmse: 30394.22833628756
mae: 18882.937383813005
r2: 0.8244977535812215


In [71]:
prd =  la1.predict(x_test)

Evaluate(prd,y_test)

mse:  1298575247.2678301
rmse: 36035.74957272056
mae: 19888.148376396955
r2: 0.770401783817181



# Ridge 

In [72]:
rd = Ridge()
rd.fit(x2,y)

Ridge()

In [73]:
tr_pred_y = rd.predict(x_train)

Evaluate(tr_pred_y,y_train)

mse:  933031772.6668197
rmse: 30545.568789381214
mae: 19045.65046219366
r2: 0.8214617918963273


In [74]:
ts_pred_y = rd.predict(x_test)

Evaluate(ts_pred_y,y_test)

mse:  1222089794.5969803
rmse: 34958.40091590261
mae: 19446.187044618247
r2: 0.7836659999459417


In [75]:
# Grid Search CV


In [76]:
gs1 = GridSearchCV(rd,param_grid=grid,cv=2)
gs1.fit(x_train,y_train)

GridSearchCV(cv=2, estimator=Ridge(),
             param_grid={'alpha': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0....
       2.21, 2.22, 2.23, 2.24, 2.25, 2.26, 2.27, 2.28, 2.29, 2.3 , 2.31,
       2.32, 2.33, 2.34, 2.35, 2.36, 2.37, 2.38, 2.39, 2.4 , 2.41, 2.42,
       2.43, 2.44, 2.45, 2.46, 2.47, 2.48, 2.49, 2.5 , 2.51, 2.52, 2.53,
       2.54, 2.55, 2.56, 2.57, 2.58, 2.59, 2.6 , 2.61, 2.62, 2.63, 2.64,
       2.65, 2.66, 2.67, 2.68, 2.69, 2.7 , 2.71, 2.72, 2.73, 2.74, 2.75,
       2.76, 2.77, 2.78, 2.79, 2.8 , 2.81, 2.82, 2.83, 2.84, 2.85, 2.86,
       2.87, 2.88, 2.89, 2.9 , 2.91, 2.92, 2.93, 2.94, 2.95, 2.96, 2.97,
       2.98, 2.99])})

In [77]:
rd1 = gs1.best_estimator_
rd1

Ridge(alpha=np.float64(2.9899999999999998))

In [78]:
tr_pred_y1 = rd1.predict(x_train)

Evaluate(tr_pred_y1,y_train)

mse:  925806887.9552685
rmse: 30427.074916187205
mae: 18909.387579578433
r2: 0.8236996591317164


In [79]:
ts_pred_y1 = rd1.predict(x_test)

Evaluate(ts_pred_y1,y_test)

mse:  1293315940.417329
rmse: 35962.70207336108
mae: 19862.970932698227
r2: 0.7708162033406496


In [81]:
rd1 = RandomizedSearchCV(rd,param_distributions=grid,cv=2)
rd1.fit(x_train,y_train)

RandomizedSearchCV(cv=2, estimator=Ridge(),
                   param_distributions={'alpha': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0....
       2.21, 2.22, 2.23, 2.24, 2.25, 2.26, 2.27, 2.28, 2.29, 2.3 , 2.31,
       2.32, 2.33, 2.34, 2.35, 2.36, 2.37, 2.38, 2.39, 2.4 , 2.41, 2.42,
       2.43, 2.44, 2.45, 2.46, 2.47, 2.48, 2.49, 2.5 , 2.51, 2.52, 2.53,
       2.54, 2.55, 2.56, 2.57, 2.58, 2.59, 2.6 , 2.61, 2.62, 2.63, 2.64,
       2.65, 2.66, 2.67, 2.68, 2.69, 2.7 , 2.71, 2.72, 2.73, 2.74, 2.75,
       2.76, 2.77, 2.78, 2.79, 2.8 , 2.81, 2.82, 2.83, 2.84, 2.85, 2.86,
       2.87, 2.88, 2.89, 2.9 , 2.91, 2.92, 2.93, 2.94, 2.95, 2.96, 2.97,
       2.98, 2.99])})

In [82]:
ra1 = rd1.best_estimator_

In [83]:
ra1

Ridge(alpha=np.float64(2.61))

In [84]:
y_pred = ra1.predict(x_train)

Evaluate(y_pred,y_train)

mse:  925580995.1771064
rmse: 30423.362654005003
mae: 18907.465315326583
r2: 0.8237864731368945


In [85]:
y_pred1 = ra1.predict(x_test)

Evaluate(y_pred1,y_test)

mse:  1293774495.8574898
rmse: 35969.07693919167
mae: 19865.96752145328
r2: 0.7708026600986737
